In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid') # sns에 흰색 그리드 유지
import missingno # 결측치 시각화

# KFold (교차 검증을 사용하기 위해)
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증 데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가 함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier


# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import  XGBRegressor

# 머신러닝 알고리즘 - 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 머신러닝 알고리즘 - 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 다중분류를 위한 원핫 인코더
from keras.utils import to_categorical

# 학습 자동 중단
from keras.callbacks import EarlyStopping

# 모델 저장
from keras.callbacks import ModelCheckpoint

# 저장된 딥러닝 모델 불러오기
from keras.models import load_model

# 딥러닝
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

from keras.utils import np_utils
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Reshape
from keras.layers import UpSampling2D
from keras.layers import Input
from keras.models import Model

# 이미지 조작을 위한
from PIL import Image

import os, glob

# 이미지 생성자
from keras.preprocessing.image import ImageDataGenerator

# VGG16 모델 (이미 학습이 완료되어 있는 이미지 인식 모델)
from keras.applications import VGG16

# 자연어 처리
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

# 전처리, 활성 함수 셋팅
from keras import optimizers, initializers, regularizers, metrics

# 저장
import pickle

# 시간 모듈
import time

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'   # 윈도우용
# plt.rcParams['font.family'] = 'AppleGothic'   # 맥용
plt.rcParams['font.size'] = 10                 # 폰트 크기
plt.rcParams['figure.figsize'] = 15,8          # 그래프 크기
plt.rcParams['axes.unicode_minus'] = False     # - 기호 깨짐 방지


# 경고 메시지가 안나오게 하기
import warnings
warnings.filterwarnings('ignore')

# gpu 사용 초기화 및 할당
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [2]:
root_dir = '../images/101_new/'

# 폴더의 이름을 가져와 결과데이터를 구성한다.
a1 = os.walk(root_dir)
categories = list(a1)[0][1]
nb_classes = len(categories)

In [3]:
# seed 설정
np.random.seed(3)
tf.random.set_seed(3)

In [4]:
# 데이터를 복원한다.
# 이미지데이터 확장전
# X_train, X_test, y_train, y_test = np.load('../images/caltech_new.npy', allow_pickle = True)

# 이미지 데이터 확장후
X_train, X_test, y_train, y_test = np.load('../images/calthec_new2.npy', allow_pickle = True)

In [5]:
# 데이터 정규화 (0~1사이로 조정)
X_train = X_train.astype('float')/255
X_test  = X_test.astype('float')/255

In [6]:
# 결과 데이터 -원핫 인코딩
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

In [7]:
# 학습 모델을 구축한다.
model = Sequential()

# 입력층 + 첫번째 은닉층
model.add(Conv2D(32, kernel_size=(3,3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 두번째 은닉층
model.add(Conv2D(64, kernel_size=(3,3), padding='same'))
model.add(Activation('relu'))

# 세번째 은닉층
model.add(Conv2D(64, kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 출력층
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 64)        3

In [9]:
# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [10]:
# 모델 저장 조건
model_path = 'models/Caltech/{epoch}-{loss}.hdf5'
callback1 = ModelCheckpoint(filepath=model_path, monitor='loss', verbose=1, save_best_only=True)


In [11]:
# 조기 중단 조건
callback2 = EarlyStopping(monitor='loss', patience=20)

In [ ]:
# 모델 학습
model.fit(X_train, y_train, batch_size=32, epochs=1000, callbacks=[callback1, callback2], verbose=0)


Epoch 00001: loss improved from inf to 1.69347, saving model to models/Caltech\1-1.6934669017791748.hdf5

Epoch 00002: loss improved from 1.69347 to 0.78640, saving model to models/Caltech\2-0.786404013633728.hdf5

Epoch 00003: loss improved from 0.78640 to 0.55772, saving model to models/Caltech\3-0.5577186346054077.hdf5

Epoch 00004: loss improved from 0.55772 to 0.46451, saving model to models/Caltech\4-0.46451398730278015.hdf5

Epoch 00005: loss improved from 0.46451 to 0.41123, saving model to models/Caltech\5-0.4112284779548645.hdf5

Epoch 00006: loss improved from 0.41123 to 0.38655, saving model to models/Caltech\6-0.3865501582622528.hdf5

Epoch 00007: loss improved from 0.38655 to 0.36719, saving model to models/Caltech\7-0.3671877086162567.hdf5

Epoch 00008: loss improved from 0.36719 to 0.35978, saving model to models/Caltech\8-0.3597817122936249.hdf5

Epoch 00009: loss improved from 0.35978 to 0.34773, saving model to models/Caltech\9-0.3477347195148468.hdf5

Epoch 00010: 

In [15]:
# 검증 데이터를 통한 평가
score = model.evaluate(X_test, y_test)
print(f'손실 : {score[0]}')
print(f'정확도 : {score[1]}')

1215/1215 [==============================] - 5s 4ms/step - loss: 0.1120 - accuracy: 0.9729
손실 : 0.11202547699213028
정확도 : 0.9728818535804749


In [16]:
# 학습 데이터를 통한 평가
score2 = model.evaluate(X_train, y_train)
print(f'손실 : {score2[0]}')
print(f'정확도 : {score2[1]}')

3644/3644 [==============================] - 15s 4ms/step - loss: 0.0124 - accuracy: 0.9967
손실 : 0.012396670877933502
정확도 : 0.9966723322868347
